In [ ]:
#!/usr/bin/env python3
import os
import logging
from pathlib import Path
import requests 
from zipfile import ZipFile

logging.basicConfig(level=logging.DEBUG)

#Import Pegasus API
from Pegasus.api import *

#Downloading dataset
zipurl = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip'    
r = requests.get(zipurl, stream = True) 
  
with open("dataset.zip","wb") as pdf: 
    for chunk in r.iter_content(chunk_size=1024): 
         if chunk: 
            pdf.write(chunk)
            
# Extract all the contents of zip file in current directory            
with ZipFile('dataset.zip', 'r') as zipObj:
    zipObj.extractall()

In [8]:
#!/usr/bin/env python3
import os
import logging
from pathlib import Path
import requests 
from glob import glob
from zipfile import ZipFile

logging.basicConfig(level=logging.DEBUG)

#Import Pegasus API
from Pegasus.api import *

input_files = glob('python/PetImages/*/*.jpg')
all_files=[]

rc = ReplicaCatalog()
for file in input_files:
    all_files.append(File(file))
    rc.add_replica("local", File(file), str(Path(".").resolve() / file))
rc.write()

#Transformation
pre_process_resize = Transformation( "Preprocessing_resize.py",
        site = "local",
        pfn = str(Path(".").resolve() / "Preprocessing_resize.py"),
        is_stageable=True)
                                    
tc = TransformationCatalog()\
    .add_transformations(pre_process_resize)\
    .write()

#Workflow
wf = Workflow("Cats_and_Dogs", infer_dependencies=True)

output_file1 = File('dogs_vs_cats_photos.npy')
outputfile2 = File('dogs_vs_cats_labels.npy')                                   

job_preprocess1 = Job(pre_process_resize)\
                    .add_inputs(*all_files)\
                    .add_outputs(output_file1,outputfile2)                         

wf.add_jobs(job_preprocess1)                                    

In [9]:
try:
    wf.plan(
        sites=["condorpool"],
        submit=True,
        staging_sites={"condorpool":"stage"}
    )
except PegasusClientError as e:
    print(e.output)

Plan:
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.07.24 14:38:57.589 UTC:    
2020.07.24 14:38:57.594 UTC:   ----------------------------------------------------------------------- 
2020.07.24 14:38:57.599 UTC:   File for submitting this DAG to HTCondor           : Cats_and_Dogs-0.dag.condor.sub 
2020.07.24 14:38:57.605 UTC:   Log of DAGMan debugging messages                 : Cats_and_Dogs-0.dag.dagman.out 
2020.07.24 14:38:57.610 UTC:   Log of HTCondor library output                